In [3]:
import os
import nibabel as nib
import numpy as np


def read_and_compare_nifti_headers(nifti1_path, nifti2_path):
    """ Read and compare the headers of two NIfTI files.
    Parameters
    Therby it lists all header fields and their types.  
    """ 
    # Load the NIfTI files
    nifti1 = nib.load(nifti1_path)
    nifti2 = nib.load(nifti2_path)

    # Get the headers
    header1 = nifti1.header
    header2 = nifti2.header

    # Get the fields of the headers
    fields1 = header1.keys()
    fields2 = header2.keys()

    print("Fields in the first header:")
    print(fields1)
    print("Fields in the second header:")
    print(fields2)

    # Compare the fields
    if fields1 != fields2:
        print("The headers have different fields.")
        print("Fields in the first header:")
        print(fields1)
        print("Fields in the second header:")
        print(fields2)
        return

    # Compare the types of the fields
    for field in fields1:
        type1 = type(header1[field])
        type2 = type(header2[field])
        if type1 != type2:
            print(f"The field {field} has different types.")
            print(f"Type in the first header: {type1}")
            print(f"Type in the second header: {type2}")
        else:
            print(f"The field {field} has the same type in both headers: {type1}")

    # Print and compare the specific fields related to the affine matrix
    affine_fields = [
        'qform_code', 'sform_code', 'quatern_b', 'quatern_c', 'quatern_d',
        'qoffset_x', 'qoffset_y', 'qoffset_z', 'srow_x', 'srow_y', 'srow_z', 'pixdim'
    ]

    for field in affine_fields:
        value1 = header1[field]
        value2 = header2[field]
        print(f"{field} in the first header: {value1}")
        print(f"{field} in the second header: {value2}")
        if np.array_equal(value1, value2):
            print(f"The field {field} is the same in both headers.")
        else:
            print(f"The field {field} is different in the headers.")

    # Print the affine matrices and their types
    affine1 = nifti1.affine
    affine2 = nifti2.affine
    print("Affine matrix of the first NIfTI file:")
    print(affine1)
    print("Type of the affine matrix of the first NIfTI file:")
    print(type(affine1))
    print("Affine matrix of the second NIfTI file:")
    print(affine2)
    print("Type of the affine matrix of the second NIfTI file:")
    print(type(affine2))

    if np.allclose(affine1, affine2):
        print("The affine matrices are almost the same.")
    else:
        print("The affine matrices are different even with np.allclose.")

    # Compare the affine matrices
    if np.array_equal(affine1, affine2):
        print("The affine matrices are the same.")
    else:
        print("The affine matrices are different.")


data_root = "/var/datasets/LIA/pazienti"
subject = "022"
nifti1_path = os.path.join(data_root, subject, subject+"_T1.nii")
nifti2_path = os.path.join(data_root, subject, subject+"_FLAIR.nii")

read_and_compare_nifti_headers(nifti1_path, nifti2_path)

Fields in the first header:
['sizeof_hdr', 'data_type', 'db_name', 'extents', 'session_error', 'regular', 'dim_info', 'dim', 'intent_p1', 'intent_p2', 'intent_p3', 'intent_code', 'datatype', 'bitpix', 'slice_start', 'pixdim', 'vox_offset', 'scl_slope', 'scl_inter', 'slice_end', 'slice_code', 'xyzt_units', 'cal_max', 'cal_min', 'slice_duration', 'toffset', 'glmax', 'glmin', 'descrip', 'aux_file', 'qform_code', 'sform_code', 'quatern_b', 'quatern_c', 'quatern_d', 'qoffset_x', 'qoffset_y', 'qoffset_z', 'srow_x', 'srow_y', 'srow_z', 'intent_name', 'magic']
Fields in the second header:
['sizeof_hdr', 'data_type', 'db_name', 'extents', 'session_error', 'regular', 'dim_info', 'dim', 'intent_p1', 'intent_p2', 'intent_p3', 'intent_code', 'datatype', 'bitpix', 'slice_start', 'pixdim', 'vox_offset', 'scl_slope', 'scl_inter', 'slice_end', 'slice_code', 'xyzt_units', 'cal_max', 'cal_min', 'slice_duration', 'toffset', 'glmax', 'glmin', 'descrip', 'aux_file', 'qform_code', 'sform_code', 'quatern_b', 

In [9]:
def update_modality(flair_path:str, t1_path: str):
    '''Todo: create new FLAIR.
    # For this, read header of original FLAIR, replace srow_x, srow_y, srow_z and the affine matric from T1. 
    # Create a new .nii file with the data from the original FLAIR and updated header and save it in the same path, using l_022_FLAIR.nii as name
    '''
    flair = nib.load(flair_path)
    t1 = nib.load(t1_path)
    header_flair = flair.header
    header_t1 = t1.header
    #print(header_flair)
    #print(header_t1)
    #print(header_flair['srow_x'])
    #print(header_t1['srow_x'])
    header_flair['srow_x'] = header_t1['srow_x']    
    header_flair['srow_y'] = header_t1['srow_y']
    header_flair['srow_z'] = header_t1['srow_z']
    header_flair['quatern_b'] = header_t1['quatern_b']
    header_flair['quatern_c'] = header_t1['quatern_c']
    header_flair['quatern_d'] = header_t1['quatern_d']
    header_flair['qoffset_x'] = header_t1['qoffset_x']
    header_flair['qoffset_y'] = header_t1['qoffset_y']
    header_flair['qoffset_z'] = header_t1['qoffset_z']

    # Set qform to the one from t


    # Create new FLAIR with updated header
    new_flair = nib.Nifti1Image(flair.get_fdata(), header=header_flair, affine=t1.affine)
    new_flair.set_qform(t1.get_qform())
    #new_flair.set_sform(t1.get_sform())
    new_flair_path = flair_path.replace('.nii', '_new.nii')
    print("Saving new FLAIR to", new_flair_path)
    nib.save(new_flair, new_flair_path)

#flair_path = "/var/datasets/LIA/022/r022_FLAIR.nii"
#t1xflair_mask_path = "/var/datasets/LIA/022/r022_ChP_mask_T1xFLAIR_manual_seg.nii"
#t1_path = "/var/datasets/LIA/022/022_T1.nii"

subject = "099"
flair_path = os.path.join("/var/datasets/LIA/pazienti", subject, subject + "_FLAIR.nii")
t1xflair_mask_path = os.path.join("/var/datasets/LIA/pazienti", subject, subject + "_ChP_mask_T1xFLAIR_manual_seg.nii")
t1_path = os.path.join("/var/datasets/LIA/pazienti", subject, subject + "_T1.nii")
print(flair_path)
print(t1_path)
print(t1xflair_mask_path)
update_modality(flair_path, t1_path)
update_modality(t1xflair_mask_path, t1_path)






/var/datasets/LIA/pazienti/099/099_FLAIR.nii
/var/datasets/LIA/pazienti/099/099_T1.nii
/var/datasets/LIA/pazienti/099/099_ChP_mask_T1xFLAIR_manual_seg.nii
Saving new FLAIR to /var/datasets/LIA/pazienti/099/099_FLAIR_new.nii
Saving new FLAIR to /var/datasets/LIA/pazienti/099/099_ChP_mask_T1xFLAIR_manual_seg_new.nii


In [45]:
base_path = "/var/datasets/LIA/"
path_021 = base_path + "Dataset011_ChoroidPlexus_T1_FLAIR_T1xFLAIRmask_sym_AP_022/"
path_022 = base_path + "022/"

t1_021_path = path_021 + "image_Tr/021_image0001.nii"
flair_021_path = path_021 + "image_Tr/021_image0000.nii"
t1xflair_seg_021_path = path_021 + "label_Tr/021seg0004.nii"

t1_022_path = path_022 + "022_T1.nii"
flair_022_path = path_022 + "r022_FLAIR_new.nii"
t1xflair_seg_022_path = path_022 + "r022_ChP_mask_T1xFLAIR_manual_seg_new.nii"

t1_021 = nib.load(t1_021_path)
flair_021 = nib.load(flair_021_path)
t1xflair_seg_021 = nib.load(t1xflair_seg_021_path)

t1_022 = nib.load(t1_022_path)
flair_022 = nib.load(flair_022_path)
t1xflair_seg_022 = nib.load(t1xflair_seg_022_path)

t1_021_header = t1_021.header
flair_021_header = flair_021.header
t1xflair_seg_021_header = t1xflair_seg_021.header

t1_022_header = t1_022.header
flair_022_header = flair_022.header
t1xflair_seg_022_header = t1xflair_seg_022.header


In [5]:
base_path = "/var/datasets/LIA/Dataset011_ChoroidPlexus_T1_FLAIR_T1xFLAIRmask_sym_AP/"
subject = "038"

t1_path = base_path + "image_Ts/"+subject+"_image0001.nii"
flair_path = base_path + "image_Ts/"+subject+"_image0000.nii"
t1xflair_seg_path = base_path + "label_Tr/"+subject+"seg0004.nii"


t1 = nib.load(t1_path)
flair = nib.load(flair_path)
#t1xflair_seg = nib.load(t1xflair_seg_path)

t1_header = t1.header
flair_header = flair.header
#t1xflair_seg_header = t1xflair_seg.header

#print(t1.get_sform())
#print(flair.get_sform())
#print(flair.get_qform())
#print(t1.get_qform())

print(np.allclose(flair.get_sform(), t1.get_sform())) #usually ok
print(np.allclose(flair.get_qform(), t1.get_qform())) #usually not ok 
print(flair.get_qform() == t1.get_qform())
print(flair.affine == t1.affine)

True
False
[[False False False  True]
 [False False False  True]
 [False False False  True]
 [ True  True  True  True]]
[[ True False False  True]
 [False  True False  True]
 [False False  True  True]
 [ True  True  True  True]]


In [82]:
print(flair_022.get_qform())
print(t1_022.get_qform())
print(flair_022.get_sform() == t1_022.get_sform())
print(flair_022.get_qform() == t1_022.get_qform())
print(flair_022.affine == t1_022.affine)

[[ 9.93404069e-01 -1.14536466e-01 -5.45465545e-03 -7.82729797e+01]
 [ 1.14540340e-01  9.93418516e-01  4.02270935e-04 -1.26848640e+02]
 [ 5.37268103e-03 -1.02439567e-03  9.99985042e-01 -1.19430756e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.93404069e-01 -1.14536466e-01 -5.45465545e-03 -7.82729797e+01]
 [ 1.14540340e-01  9.93418516e-01  4.02270935e-04 -1.26848640e+02]
 [ 5.37268103e-03 -1.02439567e-03  9.99985042e-01 -1.19430756e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [76]:
print(flair_header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 180 240 240   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.       0.999989 1.       1.       5.       0.       0.       0.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'spm - realigned'
aux_file        : b''
qform_code      : aligned
sform_code      : aligned
quatern_b       : 0.1497826
quatern_c       : -0.023198416
quatern_d       : 0.03311607
qoffset_x       : -79.44008
qoffset_

In [79]:
print(t1.get_sform())
print(flair.get_sform())
print(np.allclose(flair.get_sform(), t1.get_sform()))

[[ 9.96719301e-01 -7.23795891e-02 -3.59146595e-02 -7.25461044e+01]
 [ 5.84801435e-02  9.52937007e-01 -2.97474504e-01 -7.11170654e+01]
 [ 5.57548404e-02  2.94401526e-01  9.54053998e-01 -1.67502258e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.96719301e-01 -7.23795891e-02 -3.59146595e-02 -7.25461044e+01]
 [ 5.84801435e-02  9.52937007e-01 -2.97474504e-01 -7.11170654e+01]
 [ 5.57548404e-02  2.94401526e-01  9.54053998e-01 -1.67502258e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
True


In [75]:
print(t1_header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 54
dim             : [  3 180 240 240   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.        1.        1.        1.        0.0083276 0.        0.
 0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'TE=3.9;Time=91418.030'
aux_file        : b''
qform_code      : scanner
sform_code      : aligned
quatern_b       : 0.1497826
quatern_c       : -0.023198407
quatern_d       : 0.033116072
qoffset_x       : -79.4400

In [26]:
print(t1_021.header)
print(t1_022.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 54
dim             : [  3 180 240 240   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.        1.        1.        1.        0.0083198 0.        0.
 0.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'TE=3.8;Time=93225.680'
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : -0.029906072
quatern_c       : -0.058264695
quatern_d       : 0.05872771
qoffset_x       : -62.76

In [7]:
print(flair_021.header)
print(flair_022.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 180 240 240   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.         0.99999994 0.99999994 1.         5.         0.
 0.         0.        ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'spm - realigned'
aux_file        : b''
qform_code      : aligned
sform_code      : aligned
quatern_b       : -0.029906066
quatern_c       : -0.0582647
quatern_d       : 0.0587277
qoffset_x       : -62.76496

In [23]:
t1_021_data = t1_021.get_fdata()
t1_022_data = t1_022.get_fdata()

flair_021_data = flair_021.get_fdata()
flair_022_data = flair_022.get_fdata()

In [24]:
print(t1_021_data.dtype, flair_021_data.dtype, t1_022_data.dtype, flair_022_data.dtype)
print(t1xflair_seg_021.get_fdata().dtype, t1xflair_seg_022.get_fdata().dtype)

float64 float64 float64 float64
float64 float64


In [25]:
print(t1_021.affine.dtype)
print(flair_021.affine.dtype)

print(t1_022.affine.dtype)
print(flair_022.affine.dtype)

float64
float64
float64
float64
